In [20]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/data/", one_hot=True)

Extracting ./mnist/data/train-images-idx3-ubyte.gz
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz


In [21]:
import numpy as np
from numpy.random import randint

In [26]:
X = tf.placeholder(tf.float32, [None, 28, 28, 1])
Y = tf.placeholder(tf.float32, [None, 10])
keep_prob = tf.placeholder(tf.float32)

In [27]:
W1 = tf.Variable(tf.random_normal([3, 3, 1, 32], stddev=0.01)) # W1 커널
# 이미지의 가장 외곽에서 한 칸 밖으로 움직이는 옵션, 테두리까지도 평가함.
L1 = tf.nn.conv2d(X, W1, strides=[1, 1, 1, 1], padding='SAME')
L1 = tf.nn.relu(L1) # 활성화 함수

In [28]:
L1 = tf.nn.max_pool(L1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
print(L1)

Tensor("MaxPool_2:0", shape=(?, 14, 14, 32), dtype=float32)


In [29]:
W2 = tf.Variable(tf.random_normal([3, 3, 32, 64], stddev=0.01))
L2 = tf.nn.conv2d(L1, W2, strides=[1, 1, 1, 1], padding='SAME')
L2 = tf.nn.relu(L2)
L2 = tf.nn.max_pool(L2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

In [30]:
W3 = tf.Variable(tf.random_normal([7 * 7 * 64, 256], stddev=0.01))
L3 = tf.reshape(L2, [-1, 7 * 7 * 64])
L3 = tf.matmul(L3, W3)
L3 = tf.nn.relu(L3)
L3 = tf.nn.dropout(L3, keep_prob)
print(L3)

Tensor("dropout_1/mul_1:0", shape=(?, 256), dtype=float32)


In [31]:
# 최종 출력값 L3 에서의 출력 256개를 입력값으로 받아서
# 0~9 레이블인 10개의 출력값을 만듭니다.
W4 = tf.Variable(tf.random_normal([256, 10], stddev=0.01))
model = tf.matmul(L3, W4)
print(model)

Tensor("MatMul_3:0", shape=(?, 10), dtype=float32)


In [32]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=model,
labels=Y))
optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)

In [33]:
init = tf.global_variables_initializer() # 초기화
sess = tf.Session() # 세션 연결
sess.run(init) # 세션 초기화
batch_size = 100
total_batch = int(mnist.train.num_examples / batch_size) # 총 Total batch

In [ ]:
#결과 값 테스트 

1 dropout 비율 80%, epochs 15
2 dropout x epochs 30
3 CNN filter 5x5, stride=2
4 CNN filter 수 1단계 64, 2단계 128

소요시간
최정결과(정확도)


In [38]:
# 1 dropout 비율 80%, epochs 15

for epoch in range(15):
    total_cost = 0
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
# 이미지 데이터를 CNN 모델을 위한 자료형태인 [28 28 1] 의 형태로 재구성합니다.
        batch_xs = batch_xs.reshape(-1, 28, 28, 1)
        _, cost_val = sess.run([optimizer, cost],
        feed_dict={X: batch_xs,
                    Y: batch_ys,
                    keep_prob: 0.8})

        total_cost += cost_val
    print('Epoch:', '%04d' % (epoch + 1),
            'Avg. cost =', '{:.3f}'.format(total_cost / total_batch))
print('최적화 완료!')


is_correct = tf.equal(tf.argmax(model, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도:', sess.run(accuracy,
feed_dict={X: mnist.test.images.reshape(-1, 28, 28, 1),
Y: mnist.test.labels,
keep_prob: 1}))

Epoch: 0001 Avg. cost = nan
Epoch: 0002 Avg. cost = nan
Epoch: 0003 Avg. cost = nan
Epoch: 0004 Avg. cost = nan
Epoch: 0005 Avg. cost = nan
Epoch: 0006 Avg. cost = nan
Epoch: 0007 Avg. cost = nan
Epoch: 0008 Avg. cost = nan
Epoch: 0009 Avg. cost = nan
Epoch: 0010 Avg. cost = nan
Epoch: 0011 Avg. cost = nan
Epoch: 0012 Avg. cost = nan
Epoch: 0013 Avg. cost = nan
Epoch: 0014 Avg. cost = nan
Epoch: 0015 Avg. cost = nan
최적화 완료!
정확도: 0.098


In [37]:
# 2 dropout x epochs 30

for epoch in range(30):
    total_cost = 0
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
# 이미지 데이터를 CNN 모델을 위한 자료형태인 [28 28 1] 의 형태로 재구성합니다.
        batch_xs = batch_xs.reshape(-1, 28, 28, 1)
        _, cost_val = sess.run([optimizer, cost],
        feed_dict={X: batch_xs,
                    Y: batch_ys,
                    keep_prob: 1.0})

        total_cost += cost_val
    print('Epoch:', '%04d' % (epoch + 1),
            'Avg. cost =', '{:.3f}'.format(total_cost / total_batch))
print('최적화 완료!')

Epoch: 0001 Avg. cost = nan
Epoch: 0002 Avg. cost = nan
Epoch: 0003 Avg. cost = nan
Epoch: 0004 Avg. cost = nan
Epoch: 0005 Avg. cost = nan
Epoch: 0006 Avg. cost = nan
Epoch: 0007 Avg. cost = nan
Epoch: 0008 Avg. cost = nan
Epoch: 0009 Avg. cost = nan
Epoch: 0010 Avg. cost = nan
Epoch: 0011 Avg. cost = nan
Epoch: 0012 Avg. cost = nan
Epoch: 0013 Avg. cost = nan
Epoch: 0014 Avg. cost = nan
Epoch: 0015 Avg. cost = nan
Epoch: 0016 Avg. cost = nan
Epoch: 0017 Avg. cost = nan
Epoch: 0018 Avg. cost = nan
Epoch: 0019 Avg. cost = nan
Epoch: 0020 Avg. cost = nan
Epoch: 0021 Avg. cost = nan
Epoch: 0022 Avg. cost = nan
Epoch: 0023 Avg. cost = nan
Epoch: 0024 Avg. cost = nan
Epoch: 0025 Avg. cost = nan
Epoch: 0026 Avg. cost = nan
Epoch: 0027 Avg. cost = nan
Epoch: 0028 Avg. cost = nan
Epoch: 0029 Avg. cost = nan
Epoch: 0030 Avg. cost = nan
최적화 완료!


In [ ]:
# 3 CNN filter 5x5, stride=2

X = tf.placeholder(tf.float32, [None, 28, 28, 1])
Y = tf.placeholder(tf.float32, [None, 10])
keep_prob = tf.placeholder(tf.float32)

W1 = tf.Variable(tf.random_normal([5, 5, 1, 32], stddev=0.01)) 
L1 = tf.nn.conv2d(X, W1, strides=[1, 2, 2, 1], padding='SAME')
L1 = tf.nn.relu(L1)

W2 = tf.Variable(tf.random_normal([3, 3, 32, 64], stddev=0.01))
L2 = tf.nn.conv2d(L1, W2, strides=[1, 1, 1, 1], padding='SAME')
L2 = tf.nn.relu(L2)
L2 = tf.nn.max_pool(L2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')


W3 = tf.Variable(tf.random_normal([7 * 7 * 64, 256], stddev=0.01))
L3 = tf.reshape(L2, [-1, 7 * 7 * 64])
L3 = tf.matmul(L3, W3)
L3 = tf.nn.relu(L3)
L3 = tf.nn.dropout(L3, keep_prob)
print(L3)

W4 = tf.Variable(tf.random_normal([256, 10], stddev=0.01))
model = tf.matmul(L3, W4)
print(model)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=model,
labels=Y))
optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)


init = tf.global_variables_initializer() # 초기화
sess = tf.Session() # 세션 연결
sess.run(init) # 세션 초기화
batch_size = 100
total_batch = int(mnist.train.num_examples / batch_size) # 총 Total batch


for epoch in range(30):
    total_cost = 0
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
# 이미지 데이터를 CNN 모델을 위한 자료형태인 [28 28 1] 의 형태로 재구성합니다.
        batch_xs = batch_xs.reshape(-1, 28, 28, 1)
        _, cost_val = sess.run([optimizer, cost],
        feed_dict={X: batch_xs,
                    Y: batch_ys,
                    keep_prob: 0})

        total_cost += cost_val
    print('Epoch:', '%04d' % (epoch + 1),
            'Avg. cost =', '{:.3f}'.format(total_cost / total_batch))
print('최적화 완료!')


In [13]:
from numba import jit, cuda 
import numpy as np 

In [ ]:
# 4 CNN 컨볼루션 필터수 1단계 64, 2단계 128

X = tf.placeholder(tf.float32, [None, 28, 28, 1])
Y = tf.placeholder(tf.float32, [None, 10])
keep_prob = tf.placeholder(tf.float32)

W1 = tf.Variable(tf.random_normal([3, 3, 1, 64], stddev=0.01)) 
L1 = tf.nn.conv2d(X, W1, strides=[1, 1, 1, 1], padding='SAME')
L1 = tf.nn.relu(L1)

W2 = tf.Variable(tf.random_normal([3, 3, 32, 128], stddev=0.01))
L2 = tf.nn.conv2d(L1, W2, strides=[1, 1, 1, 1], padding='SAME')
L2 = tf.nn.relu(L2)
L2 = tf.nn.max_pool(L2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')


W3 = tf.Variable(tf.random_normal([7 * 7 * 64, 256], stddev=0.01))
L3 = tf.reshape(L2, [-1, 7 * 7 * 64])
L3 = tf.matmul(L3, W3)
L3 = tf.nn.relu(L3)
L3 = tf.nn.dropout(L3, keep_prob)
print(L3)

W4 = tf.Variable(tf.random_normal([256, 10], stddev=0.01))
model = tf.matmul(L3, W4)
print(model)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=model,
labels=Y))
optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)


init = tf.global_variables_initializer() # 초기화
sess = tf.Session() # 세션 연결
sess.run(init) # 세션 초기화
batch_size = 100
total_batch = int(mnist.train.num_examples / batch_size) # 총 Total batch



%%time
for epoch in range(30):
    total_cost = 0
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
# 이미지 데이터를 CNN 모델을 위한 자료형태인 [28 28 1] 의 형태로 재구성합니다.
        batch_xs = batch_xs.reshape(-1, 28, 28, 1)
        _, cost_val = sess.run([optimizer, cost],
        feed_dict={X: batch_xs,
                    Y: batch_ys,
                    keep_prob: 0})

        total_cost += cost_val
    print('Epoch:', '%04d' % (epoch + 1),
            'Avg. cost =', '{:.3f}'.format(total_cost / total_batch))
print('최적화 완료!')


In [11]:
%%time
for epoch in range(15):
    total_cost = 0
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
# 이미지 데이터를 CNN 모델을 위한 자료형태인 [28 28 1] 의 형태로 재구성합니다.
        batch_xs = batch_xs.reshape(-1, 28, 28, 1)
        _, cost_val = sess.run([optimizer, cost],
        feed_dict={X: batch_xs,
                    Y: batch_ys,
                    keep_prob: 0.7})

        total_cost += cost_val
    print('Epoch:', '%04d' % (epoch + 1),
            'Avg. cost =', '{:.3f}'.format(total_cost / total_batch))
print('최적화 완료!')

Epoch: 0001 Avg. cost = 0.349
Epoch: 0002 Avg. cost = 0.111
Epoch: 0003 Avg. cost = 0.078
Epoch: 0004 Avg. cost = 0.063
Epoch: 0005 Avg. cost = 0.054
Epoch: 0006 Avg. cost = 0.042
Epoch: 0007 Avg. cost = 0.037
Epoch: 0008 Avg. cost = 0.032
Epoch: 0009 Avg. cost = 0.027
Epoch: 0010 Avg. cost = 0.025
Epoch: 0011 Avg. cost = 0.022
Epoch: 0012 Avg. cost = 0.020
Epoch: 0013 Avg. cost = 0.019
Epoch: 0014 Avg. cost = 0.016
Epoch: 0015 Avg. cost = 0.015
최적화 완료!
Wall time: 11min 57s


In [ ]:
is_correct = tf.equal(tf.argmax(model, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도:', sess.run(accuracy,
feed_dict={X: mnist.test.images.reshape(-1, 28, 28, 1),
Y: mnist.test.labels,
keep_prob: 1}))